In [16]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import time
import os
from tqdm import tqdm
from geohash import encode
from sklearn.feature_extraction import DictVectorizer
from keras.layers import *
from keras.models import *

%config InlineBackend.figure_format = 'retina' # set 'png' here when working on notebook
%matplotlib inline

cache_dir='cache'
flag=True

Using TensorFlow backend.


In [2]:
train_df=pd.read_csv('data/训练数据-ccf_first_round_user_shop_behavior.csv')
test_df=pd.read_csv('data/AB榜测试集-evaluation_public.csv')
meta_df=pd.read_csv('data/训练数据-ccf_first_round_shop_info.csv')

In [3]:
# 构造线下验证集
train_df=pd.merge(train_df,meta_df,on='shop_id',how='left')
train_df=train_df[train_df.mall_id=='m_7800']
x_train=train_df[train_df.time_stamp<'2017-08-30 00:00']
x_val=train_df[train_df.time_stamp>='2017-08-30 00:00']

In [6]:
x_train.head()

,user_id,shop_id,time_stamp,longitude_x,latitude_x,wifi_infos,category_id,longitude_y,latitude_y,price,mall_id
39,u_9611,s_684235,2017-08-05 15:30,109.264884,35.258471,b_31569692|-83|false;b_45321577|-82|false;b_38...,c_28,109.265016,35.258217,50,m_7800
40,u_9611,s_539726,2017-08-05 15:40,109.265270,35.258223,b_45321577|-71|false;b_45323103|-74|false;b_45...,c_13,109.265325,35.258245,52,m_7800
41,u_9611,s_684235,2017-08-05 18:10,109.264854,35.258351,b_42964305|-83|false;b_55544018|-53|false;b_43...,c_28,109.265016,35.258217,50,m_7800
279,u_48238,s_685058,2017-08-23 12:10,109.265269,35.258723,b_45322897|-61|false;b_14364825|-59|false;b_26...,c_28,109.265016,35.258217,44,m_7800
280,u_48238,s_992598,2017-08-23 12:10,109.265222,35.258499,b_44663058|-78|false;b_18983684|-60|false;b_40...,c_30,109.265815,35.257716,45,m_7800


In [89]:
def make_wifi_dict(x):
    def _transform(x):
        i=x.split('|')[:2]
        i[1]=int(i[1])
        return i
    result=[_transform(i) for i in x.split(';')]
    return dict(result)
train_wifi=x_train.wifi_infos.apply(make_wifi_dict)
val_wifi=x_val.wifi_infos.apply(make_wifi_dict)

In [90]:
vec = DictVectorizer()
train_vec = vec.fit_transform(list(train_wifi)).toarray()
val_vec = vec.transform(list(val_wifi)).toarray()
train_vec.shape,val_vec.shape

((18674, 3498), (1058, 3498))

In [91]:
train_vec[train_vec==0]=-150
val_vec[val_vec==0]=-150

In [21]:
# 标准化
_min=np.min([np.min(train_vec),np.min(val_vec)])
train_vec/=_min
val_vec/=_min

In [92]:
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()
train_vec=scaler.fit_transform(train_vec)
val_vec=scaler.transform(val_vec)

In [93]:
input_dim=train_vec.shape[1]

input_layer=Input((input_dim,))
m=Dense(512,activation='relu')(input_layer)
encoder=Dense(256,activation='tanh')(m)
m=Dense(512,activation='relu')(encoder)
m=Dense(input_dim,activation='linear')(m)
ae=Model(input_layer,m)
ae.compile(optimizer='adam', loss='mse')
ae.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        (None, 3498)              0         
_________________________________________________________________
dense_48 (Dense)             (None, 512)               1791488   
_________________________________________________________________
dense_49 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_50 (Dense)             (None, 512)               131584    
_________________________________________________________________
dense_51 (Dense)             (None, 3498)              1794474   
Total params: 3,848,874
Trainable params: 3,848,874
Non-trainable params: 0
_________________________________________________________________


In [94]:
ae.fit(train_vec,train_vec,epochs=100,batch_size=256,validation_data=(val_vec,val_vec))

Train on 18674 samples, validate on 1058 samples
Epoch 1/100
18674/18674 [==============================] - 5s - loss: 0.9927 - val_loss: 0.5505
Epoch 2/100
18674/18674 [==============================] - 4s - loss: 0.9478 - val_loss: 0.5181
Epoch 3/100
18674/18674 [==============================] - 4s - loss: 0.9085 - val_loss: 0.5042ss: 0.90 - ETA: 0s - loss
Epoch 4/100
18674/18674 [==============================] - 4s - loss: 0.8794 - val_loss: 0.4973
Epoch 5/100
18674/18674 [==============================] - 4s - loss: 0.8581 - val_loss: 0.4931
Epoch 6/100
18674/18674 [==============================] - 4s - loss: 0.8418 - val_loss: 0.4906
Epoch 7/100
18674/18674 [==============================] - 4s - loss: 0.8284 - val_loss: 0.4897
Epoch 8/100
18674/18674 [==============================] - 4s - loss: 0.8173 - val_loss: 0.4897
Epoch 9/100
18674/18674 [==============================] - 4s - loss: 0.8073 - val_loss: 0.4895
Epoch 10/100
18674/18674 [==============================] - 4s

KeyboardInterrupt: 

In [81]:
en=Model(input_layer,encoder)
en.predict(train_vec)

array([[ 0.79962045, -0.92479318,  0.86232191, ..., -0.51805621,
        -0.79050088,  0.11023718],
       [ 0.28795418, -0.44653812, -0.17220461, ..., -0.02667675,
        -0.34625056, -0.61891091],
       [ 0.93289393, -0.79069847,  0.72056103, ..., -0.82164705,
         0.85884899, -0.08806316],
       ..., 
       [ 0.97734338, -0.00942533, -0.88781101, ..., -0.71490544,
        -0.96233052,  0.53717208],
       [ 0.44199413, -0.29273978, -0.08979679, ..., -0.0356275 ,
        -0.0608784 , -0.30264673],
       [ 0.98221242, -0.92761981, -0.88067973, ..., -0.59375298,
         0.90352499, -0.77659899]], dtype=float32)

In [82]:
l=[]
for i in train_df.shop_id:
    l.append({i:1})
label=DictVectorizer().fit_transform(l).toarray()
train_label=label[:x_train.shape[0],:]
val_label=label[x_train.shape[0]:,:]

In [83]:
train_label.shape,val_label.shape

((18674, 89), (1058, 89))

In [87]:
mm=Dense(256,activation='relu')(encoder)
mm=Dropout(0.5)(mm)
mm=Dense(train_label.shape[1],activation='softmax')(mm)
cls=Model(input_layer,mm)
cls.compile(optimizer='adam', loss='categorical_crossentropy')
for i in cls.layers[:3]:
    i.trainable=False
cls.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 3498)              0         
_________________________________________________________________
dense_38 (Dense)             (None, 512)               1791488   
_________________________________________________________________
dense_39 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_46 (Dense)             (None, 256)               65792     
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_47 (Dense)             (None, 89)                22873     
Total params: 2,011,481
Trainable params: 88,665
Non-trainable params: 1,922,816
_____________________________________________________________

In [88]:
cls.fit(train_vec,train_label,epochs=100,batch_size=256,validation_data=(val_vec,val_label))

Train on 18674 samples, validate on 1058 samples
Epoch 1/100
18674/18674 [==============================] - 1s - loss: 4.2672 - val_loss: 4.0807
Epoch 2/100
18674/18674 [==============================] - 0s - loss: 4.0213 - val_loss: 4.0328
Epoch 3/100
18674/18674 [==============================] - 0s - loss: 3.9477 - val_loss: 4.0278
Epoch 4/100
18674/18674 [==============================] - 0s - loss: 3.9066 - val_loss: 4.0192
Epoch 5/100
18674/18674 [==============================] - 0s - loss: 3.8707 - val_loss: 4.0233
Epoch 6/100
18674/18674 [==============================] - 0s - loss: 3.8337 - val_loss: 4.0183
Epoch 7/100
18674/18674 [==============================] - 0s - loss: 3.8063 - val_loss: 4.0223
Epoch 8/100
18674/18674 [==============================] - 0s - loss: 3.7733 - val_loss: 4.0223
Epoch 9/100
18674/18674 [==============================] - 0s - loss: 3.7522 - val_loss: 4.0265
Epoch 10/100
18674/18674 [==============================] - 0s - loss: 3.7321 - val_los

KeyboardInterrupt: 

In [50]:
ans=cls.predict(val_vec).argmax(1)
ans

NameError: name 'cls' is not defined

In [136]:
pd.Series(ans).sort_values()

1279    63
2143    63
4457    63
3542    63
2316    63
1969    64
3239    71
3241    71
3242    71
3250    71
3243    71
3244    71
3238    71
3245    71
3246    71
3247    71
3248    71
3249    71
3240    71
3251    71
0       71
3237    71
3268    71
3267    71
3266    71
3265    71
3264    71
3263    71
3262    71
3252    71
        ..
1606    71
1605    71
1604    71
1603    71
1602    71
1601    71
1600    71
1599    71
1614    71
1615    71
1616    71
1617    71
1633    71
1632    71
1631    71
1630    71
1629    71
1628    71
1627    71
1825    71
1626    71
1624    71
1623    71
1622    71
1621    71
1620    71
1619    71
1618    71
1625    71
4872    71
dtype: int64